In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import os
import time
import json

from darts.allocation import Allocator
from darts.bandit import Bandit

In [2]:
# load in the data for the target pool
start = time.time()
file_loc = 'data/'
allocation_pool_df = pd.read_csv(file_loc + 'target_voter_universe.csv')
print(f"... completed in {time.time() - start} seconds")

... completed in 0.05939030647277832 seconds


In [3]:
allocation_pool_df.head(10)

,target_id,model_id,probability,target_round,target_result,target_reward,target_regret
0,0,model1_rf,0.0,0,0,0,0
1,0,model2_lr,1.0,0,0,0,0
2,0,model3_lr,0.0,0,0,0,0
3,0,model4_xgb,1.0,0,0,0,0
4,0,model5_lgbm,0.0,0,0,0,0
5,0,model_baseline,1.0,0,0,0,0
6,1,model1_rf,0.0,0,0,0,0
7,1,model2_lr,0.0,0,0,0,0
8,1,model3_lr,0.0,0,0,0,0
9,1,model4_xgb,0.0,0,0,0,0


In [4]:
(allocation_pool_df.shape[0]/6)/250

54.54

In [5]:
# load in our target results
start = time.time()
file_loc = 'data/'
target_results_df = pd.read_csv(file_loc + 'voter_responses.csv')
print(f"... completed in {time.time() - start} seconds")

... completed in 0.009937524795532227 seconds


In [6]:
target_results_df.head(10)

,target_id,target_result
0,0,1
1,1,0
2,3,0
3,4,0
4,16,1
5,22,0
6,26,0
7,27,1
8,29,0
9,31,0


In [7]:
# Specify the column that indicates the id of an individual target
target_id_col = 'target_id'

# Specify the column that provides the confidence ranking (e.g. probability)
# that any of the models have on this target resulting in a 'reward'
# (e.g. the target outcome of the model)
confidence_rank_col = 'probability'

# Specify the column containing the names of the different 'arms' or
# models we're using in the target pool
arm_name_col = 'model_id'

# Specify column to track the pool round
pool_round_col = 'target_round'

# Specify the column that indicates the 'result'
result_col = 'target_result'

# Specify the column that indicates the 'reward'
reward_col = 'target_reward'

# Specify the column that indicates the 'regret'
regret_col = 'target_regret'

# Specify the column that indicates which arm was 'picked'
picked_col = 'model_picked'

# Specify an initial allocation method across the pool of models. These must sum to 1.
# For the simulation, we'll start with an equal allocation for each model.
allocation_method = {
    'model_baseline': (1/6),
    'model1_rf': (1/6),
    'model2_lr': (1/6),
    'model3_lr': (1/6),
    'model4_xgb': (1/6),
    'model5_lgbm': (1/6)
}

# Specify number of rounds to simulate
simulation_rounds = 50

# Specify the number of targets to pull in the first round.
num_targets = 250

# Specify the allocation policy
allocation_policy = 'UCB1'

# Specify the allocation strategy
allocation_strategy = 'round-robin'

# Specify the allocation order
allocation_order = 'best'

# bandit parameters
ucb_scale = 1.96
epsilon = 0.1
greed_factor = 1

In [8]:
# Index the file of responses for updating of results
target_results_df = target_results_df.set_index([target_id_col])

In [9]:
# Set up defaults - nothing picked yet
allocation_pool_df[picked_col]  = 0

timesteps = {}

for allocation_round in range(1, simulation_rounds+1):
    
    remaining_allocation_pool_df = pd.DataFrame.copy(allocation_pool_df[allocation_pool_df[pool_round_col]==0])

    # set up an allocator
    # TODO: I think this should be in the sequence of:
    #  allocation_pool_df, arm_name_col, target_id_col, confidence_rank_col, allocation_method, strategy, order
    # TODO: Would be good to explain that Allocator introduces a picked column to the dataframe. This is masked behavior.
    #       I may rely on this column if I know about it, but not sure if it will ever go away?
    allocator = Allocator(allocation_method, num_targets, remaining_allocation_pool_df, arm_name_col,
                          confidence_rank_col, target_id_col, strategy=allocation_strategy,
                          order=allocation_order) # another idea is to provide a dictionary mapping to simplify the calling interface

    # retreive targets
    targets = allocator.allocate_pool() # TODO: can we change this to allocator.retrieve_targets(num_targets) ? Also, might be nice to have a DF return option

    # convert to dataframe
    targets_df = pd.DataFrame(targets, columns=[target_id_col, arm_name_col])
    targets_df[pool_round_col] = allocation_round
    targets_df = targets_df[[target_id_col, pool_round_col]]

    # join targets with universe of allocations
    targets_df = targets_df.set_index([target_id_col])
    allocation_pool_df = allocation_pool_df.set_index([target_id_col])
    allocation_pool_df.update(targets_df)
    allocation_pool_df = allocation_pool_df.reset_index()   

    # add indicator for the arm we picked
    target_arm_picked_df = pd.DataFrame(targets, columns=[target_id_col, arm_name_col])
    target_arm_picked_df[picked_col] = 1
    target_arm_picked_df = target_arm_picked_df[[target_id_col, arm_name_col, picked_col]]

    # join target model we picked with the universe of allocations
    target_arm_picked_df = target_arm_picked_df.set_index([target_id_col, arm_name_col])
    allocation_pool_df = allocation_pool_df.set_index([target_id_col, arm_name_col])
    allocation_pool_df.update(target_arm_picked_df)
    allocation_pool_df = allocation_pool_df.reset_index()

    # evaluate this allocation pool with the bandit
    last_allocation_pool_df = allocation_pool_df[allocation_pool_df[pool_round_col] == allocation_round]

    # sync up results
    last_allocation_pool_df = last_allocation_pool_df.set_index([target_id_col])
    last_allocation_pool_df.update(target_results_df)
    last_allocation_pool_df = last_allocation_pool_df.reset_index(level=0)

    # update results in the master pool set
    allocation_pool_df = allocation_pool_df.set_index([target_id_col, arm_name_col, confidence_rank_col, pool_round_col, reward_col, regret_col, picked_col])
    last_allocation_pool_df = last_allocation_pool_df.set_index([target_id_col, arm_name_col, confidence_rank_col, pool_round_col, reward_col, regret_col, picked_col])
    allocation_pool_df.update(last_allocation_pool_df)
    allocation_pool_df = allocation_pool_df.reset_index()
    
    TP = allocation_pool_df[result_col] * allocation_pool_df[confidence_rank_col] * allocation_pool_df[picked_col]
    TN = (1 - allocation_pool_df[result_col]) * (1 - allocation_pool_df[confidence_rank_col]) * (1 - allocation_pool_df[picked_col])
    FN = allocation_pool_df[result_col] * (1 - allocation_pool_df[confidence_rank_col]) * (1 - allocation_pool_df[picked_col])
    FP = (1 - allocation_pool_df[result_col]) * allocation_pool_df[confidence_rank_col] * allocation_pool_df[picked_col]
    
    # update rewards from the results we synced
    allocation_pool_df[reward_col] = TP

    # update regrets from the results we synced
    # regrets can be false positives or false negatives (type 1 and type 2 error)
    allocation_pool_df[regret_col] = FN

    # prepare a dataframe with just our results of this round for the bandit to evaluate
    results_df = pd.DataFrame.copy(allocation_pool_df[allocation_pool_df[pool_round_col] == allocation_round])
    
    if results_df[(results_df[result_col]==1)&(results_df[reward_col]==1)].shape[0] == 0:
        # no more results left after a steady state - reset selections to include possibilities from the other models

        TP = allocation_pool_df[result_col] * allocation_pool_df[confidence_rank_col]
        TN = (1 - allocation_pool_df[result_col]) * (1 - allocation_pool_df[confidence_rank_col])
        FN = allocation_pool_df[result_col] * (1 - allocation_pool_df[confidence_rank_col])
        FP = (1 - allocation_pool_df[result_col]) * allocation_pool_df[confidence_rank_col]

        # update rewards from the results we synced
        allocation_pool_df[reward_col] = TP

        # update regrets from the results we synced
        # regrets can be false positives or false negatives (type 1 and type 2 error)
        allocation_pool_df[regret_col] = FN

        # prepare a dataframe with just our results of this round for the bandit to evaluate
        results_df = pd.DataFrame.copy(allocation_pool_df[allocation_pool_df[pool_round_col] == allocation_round])
        
    
    # prepare dictionary of results for timestep tracking
    stats = results_df[[arm_name_col, reward_col, regret_col]].groupby(arm_name_col).agg({reward_col:['count','sum'],
                                                                                          regret_col:['sum']})
    stats.columns = stats.columns.to_flat_index()
    stats.columns = ['_'.join(tup).rstrip('_') for tup in stats.columns.values]
    stats = pd.DataFrame(stats).reset_index()
    stats.rename(columns={'target_reward_count': 'total_pool_size', 'target_reward_sum': 'rewards', 'target_regret_sum':'regrets'}, inplace=True)
    stats = stats.set_index([arm_name_col])
    stats['allocation'] = 0.0
    allocs = pd.DataFrame.from_dict(allocation_method, orient='index', columns = ['allocation'])
    allocs.index.name=arm_name_col
    stats.update(allocs)
    stats = stats.reset_index()
    timesteps[allocation_round] = stats.to_dict(orient='records')

    # set up a multi-arm bandit and calculate allocations to each arm.
    bandit = Bandit(results_df, arm_name_col, reward_col, regret_col, policy = allocation_policy, t = allocation_round, ucb_scale = ucb_scale, epsilon = epsilon, greed_factor = greed_factor)    

    # use these allocations for the next round
    allocation_method = bandit.get_new_allocations()# bandit.make_allocs().set_index(arm_name_col)['allocation'].to_dict()
    print("Allocations after round", allocation_round, ":\n", allocation_method)
    display(bandit.get_allocation_stats())

    # useful for outputting the result of each timestep into its own directory
    # and keeping an updated master file
    
    #allocation_pool_df[[target_id_col, arm_name_col, confidence_rank_col, pool_round_col, reward_col, result_col]].sort_values(by=[target_id_col,arm_name_col]).to_csv(file_loc + 'simulated_target_universe_updates.csv', index=False)
    
    #round_loc = file_loc + 'round_{:05}/'.format(allocation_round)
    #if not os.path.exists(round_loc):
    #    os.makedirs(round_loc)
    #    
    #targets_df.sort_values(by=[target_id_col])[target_id_col].to_csv(round_loc + 'targets.csv', index=False)
    #allocation_pool_df[[target_id_col, arm_name_col, confidence_rank_col, pool_round_col, reward_col, result_col]].sort_values(by=[target_id_col,arm_name_col]).to_csv(round_loc + 'simulated_allocation_round_{:05}'.format(allocation_round) +'.csv', index=False)
    

Allocations after round 1 :
 {'model3_lr': 0.23076923076923075, 'model_baseline': 0.1923076923076923, 'model2_lr': 0.17307692307692304, 'model4_xgb': 0.16346153846153846, 'model5_lgbm': 0.12499999999999999, 'model1_rf': 0.11538461538461538}


,model_id,target_reward_mean,target_reward_count,target_reward_std,target_reward_sum,target_regret_mean,target_regret_count,target_regret_std,target_regret_sum,score,exp_score,allocation
2,model3_lr,0.096,250,0.295182,24.0,0.224,250,0.417758,56.0,0.096,0.096,0.230769
5,model_baseline,0.080,250,0.271837,20.0,0.212,250,0.409545,53.0,0.080,0.080,0.192308
1,model2_lr,0.072,250,0.259006,18.0,0.176,250,0.381584,44.0,0.072,0.072,0.173077
3,model4_xgb,0.068,250,0.252251,17.0,0.040,250,0.196352,10.0,0.068,0.068,0.163462
4,model5_lgbm,0.052,250,0.222472,13.0,0.156,250,0.363583,39.0,0.052,0.052,0.125000
0,model1_rf,0.048,250,0.214195,12.0,0.212,250,0.409545,53.0,0.048,0.048,0.115385


Allocations after round 2 :
 {'model3_lr': 0.23874416193769316, 'model2_lr': 0.17018264204574113, 'model4_xgb': 0.15963471590851774, 'model_baseline': 0.15436075283990605, 'model1_rf': 0.13853886363407097, 'model5_lgbm': 0.13853886363407097}


,model_id,target_reward_mean,target_reward_count,target_reward_std,target_reward_sum,target_regret_mean,target_regret_count,target_regret_std,target_regret_sum,score,exp_score,allocation
2,model3_lr,0.132,250,0.339170,33.0,0.260,250,0.439514,65.0,0.181074,0.181074,0.238744
1,model2_lr,0.080,250,0.271837,20.0,0.184,250,0.388261,46.0,0.129074,0.129074,0.170183
3,model4_xgb,0.072,250,0.259006,18.0,0.052,250,0.222472,13.0,0.121074,0.121074,0.159635
5,model_baseline,0.068,250,0.252251,17.0,0.236,250,0.425474,59.0,0.117074,0.117074,0.154361
0,model1_rf,0.056,250,0.230383,14.0,0.200,250,0.400802,50.0,0.105074,0.105074,0.138539
4,model5_lgbm,0.056,250,0.230383,14.0,0.168,250,0.374616,42.0,0.105074,0.105074,0.138539


Allocations after round 3 :
 {'model3_lr': 0.19755058046645751, 'model2_lr': 0.1775177715152418, 'model5_lgbm': 0.16249316480183007, 'model_baseline': 0.16249316480183007, 'model4_xgb': 0.15748496256402614, 'model1_rf': 0.14246035585061437}


...

,model_id,target_reward_mean,target_reward_count,target_reward_std,target_reward_sum,target_regret_mean,target_regret_count,target_regret_std,target_regret_sum,score,exp_score,allocation
2,model3_lr,0.108,250,0.311003,27.0,0.248,250,0.432718,62.0,0.223974,0.223974,0.202171
4,model5_lgbm,0.072,250,0.259006,18.0,0.168,250,0.374616,42.0,0.187974,0.187974,0.169676
3,model4_xgb,0.064,250,0.245244,16.0,0.060,250,0.237963,15.0,0.179974,0.179974,0.162454
1,model2_lr,0.060,250,0.237963,15.0,0.228,250,0.420384,57.0,0.175974,0.175974,0.158844
5,model_baseline,0.060,250,0.237963,15.0,0.228,250,0.420384,57.0,0.175974,0.175974,0.158844
0,model1_rf,0.048,250,0.214195,12.0,0.208,250,0.406691,52.0,0.163974,0.163974,0.148012


Allocations after round 49 :
 {'model3_lr': 0.19994966896980298, 'model4_xgb': 0.17855345320350105, 'model1_rf': 0.1642893093592998, 'model_baseline': 0.15715723743719914, 'model2_lr': 0.1500251655150985, 'model5_lgbm': 0.1500251655150985}


,model_id,target_reward_mean,target_reward_count,target_reward_std,target_reward_sum,target_regret_mean,target_regret_count,target_regret_std,target_regret_sum,score,exp_score,allocation
2,model3_lr,0.108,250,0.311003,27.0,0.260,250,0.439514,65.0,0.224282,0.224282,0.199950
3,model4_xgb,0.084,250,0.277944,21.0,0.060,250,0.237963,15.0,0.200282,0.200282,0.178553
0,model1_rf,0.068,250,0.252251,17.0,0.204,250,0.403777,51.0,0.184282,0.184282,0.164289
5,model_baseline,0.060,250,0.237963,15.0,0.264,250,0.441684,66.0,0.176282,0.176282,0.157157
1,model2_lr,0.052,250,0.222472,13.0,0.220,250,0.415077,55.0,0.168282,0.168282,0.150025
4,model5_lgbm,0.052,250,0.222472,13.0,0.152,250,0.359741,38.0,0.168282,0.168282,0.150025


Allocations after round 50 :
 {'model1_rf': 0.17750185935610696, 'model2_lr': 0.17750185935610696, 'model5_lgbm': 0.17027839756314678, 'model3_lr': 0.16305493577018657, 'model4_xgb': 0.15583147397722638, 'model_baseline': 0.15583147397722638}


,model_id,target_reward_mean,target_reward_count,target_reward_std,target_reward_sum,target_regret_mean,target_regret_count,target_regret_std,target_regret_sum,score,exp_score,allocation
0,model1_rf,0.080,250,0.271837,20.0,0.196,250,0.397765,49.0,0.196584,0.196584,0.177502
1,model2_lr,0.080,250,0.271837,20.0,0.180,250,0.384958,45.0,0.196584,0.196584,0.177502
4,model5_lgbm,0.072,250,0.259006,18.0,0.152,250,0.359741,38.0,0.188584,0.188584,0.170278
2,model3_lr,0.064,250,0.245244,16.0,0.264,250,0.441684,66.0,0.180584,0.180584,0.163055
3,model4_xgb,0.056,250,0.230383,14.0,0.056,250,0.230383,14.0,0.172584,0.172584,0.155831
5,model_baseline,0.056,250,0.230383,14.0,0.208,250,0.406691,52.0,0.172584,0.172584,0.155831


In [10]:
timesteps

{1: [{'model_id': 'model1_rf',
   'total_pool_size': 250,
   'rewards': 12.0,
   'regrets': 53.0,
   'allocation': 0.16666666666666666},
  {'model_id': 'model2_lr',
   'total_pool_size': 250,
   'rewards': 18.0,
   'regrets': 44.0,
   'allocation': 0.16666666666666666},
  {'model_id': 'model3_lr',
   'total_pool_size': 250,
   'rewards': 24.0,
   'regrets': 56.0,
   'allocation': 0.16666666666666666},
  {'model_id': 'model4_xgb',
   'total_pool_size': 250,
   'rewards': 17.0,
   'regrets': 10.0,
   'allocation': 0.16666666666666666},
  {'model_id': 'model5_lgbm',
   'total_pool_size': 250,
   'rewards': 13.0,
   'regrets': 39.0,
   'allocation': 0.16666666666666666},
  {'model_id': 'model_baseline',
   'total_pool_size': 250,
   'rewards': 20.0,
   'regrets': 53.0,
   'allocation': 0.16666666666666666}],
 2: [{'model_id': 'model1_rf',
   'total_pool_size': 250,
   'rewards': 14.0,
   'regrets': 50.0,
   'allocation': 0.11538461538461538},
  {'model_id': 'model2_lr',
   'total_pool_size

In [11]:
# write simulation sequence to a json file
with open('timesteps.json', 'w') as json_file:
    json.dump(timesteps, json_file)